In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

In [2]:
from core.io.loader import *
from features.authors import *
from features.collaboration import *

In [3]:
from plot.boxplot import *

In [4]:
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline

In [6]:
train = get_data_with_node_information('train', data_dir='../data')

In [7]:
train = ExtractAuthorsList().transform(train)

In [9]:
collaboration_shortests_paths = CollaborationDistance()

In [10]:
train = collaboration_shortests_paths.transform(train)

In [12]:
class CollaborationFeatures:

    def fit(self, df):
        pass

    def transform(self, df):

        df['common_authors'] = df.collaboration_distance.apply(lambda x: number_of_occurences(x, 0))
        df['authors_collaboration'] = \
            df.collaboration_distance.apply(lambda x: number_of_occurences(x, 1))
        df['collaboration_min_distance'] = \
            df.collaboration_distance.apply(lambda x: np.min(x) if x is not None else np.inf)
        df['collaboration_mean_distance'] = \
            df.collaboration_distance.apply(lambda x: np.mean(x) if x is not None else np.inf)

        df['writer_in_target_paper'] = \
            df.writer_collaboration_distance.apply(lambda x: isinstance(x, list) and 0 in x)
        df['writer_collaborators'] = \
            df.writer_collaboration_distance.apply(lambda x: number_of_occurences(x, 1))
        df['writer_collaboration_min_distance'] = \
            df.writer_collaboration_distance.apply(lambda x: np.min(x) if x is not None else np.inf)
        df['writer_collaboration_mean_distance'] = \
            df.writer_collaboration_distance.apply(lambda x: np.mean(x) if x is not None else np.inf)

        return df

In [13]:
train = CollaborationFeatures().transform(train)

In [14]:
train

,source_id,target_id,category,source_publication_date,target_publication_date,source_title,target_title,source_authors,target_authors,source_journal,...,collaboration_distance,writer_collaboration_distance,common_authors,authors_collaboration,collaboration_min_distance,collaboration_mean_distance,writer_in_target_paper,writer_collaborators,writer_collaboration_min_distance,writer_collaboration_mean_distance
0,9510123,9502114,1,1995,1995,an infinite number of potentials surrounding 2...,stability analysis of the dilatonic black hole...,NaN,"Won T. Kim, Julian Lee, , Young Jai Park",Phys.Lett.,...,None,None,0,0,inf,inf,False,0,inf,inf
510736,9604093,9502114,1,1996,1995,quantisation of 2d-gravity with weyl and area-...,stability analysis of the dilatonic black hole...,"J.-G. Zhou, Y.-G. Miao, J.-Q.Liang, H.J.W.Muel...","Won T. Kim, Julian Lee, , Young Jai Park",Z.Phys.,...,"[5.0, 6.0, 5.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...","[5.0, 6.0, 5.0]",0,0,4.000000,4.733333,False,0,5.000000,5.333333
451434,9601150,9502114,0,1996,1995,duality of type ii 7-branes and 8-branes,stability analysis of the dilatonic black hole...,"E. Bergshoeff, M. de Roo, M.B. Green, G. Papad...","Won T. Kim, Julian Lee, , Young Jai Park",Nucl.Phys.,...,"[5.0, 4.0, 5.0, 6.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[5.0, 4.0, 5.0]",0,0,4.000000,5.000000,False,0,4.000000,4.666667
168455,9703014,9502114,0,1997,1995,quasi-topological field theories in two dimens...,stability analysis of the dilatonic black hole...,"Bruno G. Carneiro da Cunha, P. Teotonio-Sobrinho","Won T. Kim, Julian Lee, , Young Jai Park",Int.J.Mod.Phys.,...,"[7.0, 7.0, 6.0, 6.0, 6.0, 5.0]","[7.0, 7.0, 6.0]",0,0,5.000000,6.166667,False,0,6.000000,6.666667
170930,9603017,9502114,1,1996,1995,a new look at the rst model,stability analysis of the dilatonic black hole...,"Jian-Ge Zhou, F. Zimmerschied, J.--Q. Liang, H...","Won T. Kim, Julian Lee, , Young Jai Park",Mod.Phys.Lett.,...,"[4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...","[4.0, 4.0, 3.0]",0,0,3.000000,4.250000,False,0,3.000000,3.666667
21430,9509112,9502114,1,1995,1995,2d quantum dilaton gravitational hamiltonian b...,stability analysis of the dilatonic black hole...,"H.J.W.Muller-Kirsten, Jian-Ge Zhou, Yan-Gang M...","Won T. Kim, Julian Lee, , Young Jai Park",Phys.Lett.,...,"[4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, ...","[4.0, 4.0, 3.0]",0,0,3.000000,3.833333,False,0,3.000000,3.666667
426916,9607084,9502114,0,1996,1995,gauging of lorentz group wzw model by its null...,stability analysis of the dilatonic black hole...,Amir Masoud Ghezelbash,"Won T. Kim, Julian Lee, , Young Jai Park",Mod.Phys.Lett.,...,"[5.0, 4.0, 5.0]","[5.0, 4.0, 5.0]",0,0,4.000000,4.666667,False,0,4.000000,4.666667
201084,9404020,9502114,0,1994,1995,renormalization group effects in the conformal...,stability analysis of the dilatonic black hole...,"S. Odintsov, R. Percacci","Won T. Kim, Julian Lee, , Young Jai Park",Mod.Phys.Lett.,...,"[6.0, 5.0, 6.0, 5.0, 4.0, 5.0]","[6.0, 5.0, 6.0]",0,0,4.000000,5.166667,False,0,5.000000,5.666667
167792,9507085,9502114,0,1995,1995,on the uniqueness of the twisted representatio...,stability analysis of the dilatonic black hole...,NaN,"Won T. Kim, Julian Lee, , Young Jai Park",Nucl.Phys.,...,None,None,0,0,inf,inf,False,0,inf,inf
192101,9609145,9502114,0,1996,1995,the effective prepotential of n 2 supersymmetr...,stability analysis of the dilatonic black hole...,"Eric D'Hoker, I. M. Krichever, D. H. Phong","Won T. Kim, Julian Lee, , Young Jai Park",Nucl.Phys.,...,"[5.0, 4.0, 4.0, 6.0, 5.0, 5.0, 6.0, 5.0, 5.0]","[5.0, 4.0, 4.0]",0,0,4.000000,5.000000,False,0,4.000000,4.333333


In [72]:
positive = train.category == 1

In [73]:
negative = train.category == 0

###### Number of common authors

In [74]:
def number_of_occurences(l, x):
    cpt = 0
    if isinstance(l, list):
        for element in l:
            if element == x:
                cpt+=1
    return cpt

In [75]:
train['common_authors'] = train.collaboration_paths.apply(lambda x: number_of_occurences(x, 0))

###### Number of pair of authors that have collaborated

In [76]:
train['authors_collaboration'] = train.collaboration_paths.apply(lambda x: number_of_occurences(x, 1))

###### Minimum distance between authors 

In [77]:
train['collaboration_min_distance'] = train.collaboration_paths.apply(lambda x: np.min(x) if x is not None else np.inf)

In [78]:
train['collaboration_mean_distance'] = train.collaboration_paths.apply(lambda x: np.mean(x) if x is not None else np.inf)

### Writer collaboration

In [79]:
class WriterCollaborationShortestPaths(CollaborationShortestPaths):
    
    def transform(self, df):
        self.collaboration_graph, self.authors_mapping = create_collaboration_graph(df)
        self.shortest_paths_matrix = extract_shortest_paths_matrix(self.collaboration_graph)
        authors_list = \
            df.source_authors_list.apply(lambda x: [[x[0]]] if (isinstance(x, list) and len(x) > 0) else None) + \
            df.target_authors_list.apply(lambda x: [x] if x is not None else None)
        df['writer_collaboration_paths'] = authors_list.apply(
            lambda x: extract_shortest_paths(x, self.shortest_paths_matrix, self.authors_mapping)
        )
        return df

In [80]:
train = WriterCollaborationShortestPaths().transform(train)

###### Writer in source and target

In [85]:
train['writer_in_target_paper'] = train.writer_collaboration_paths.apply(lambda x: isinstance(x, list) and 0 in x)

In [103]:
(train.writer_in_target_paper > 0).value_counts()


False    590620
True      24892
Name: writer_in_target_paper, dtype: int64

In [104]:
train[train.writer_in_target_paper > 0].category.value_counts()

1    24802
0       90
Name: category, dtype: int64

###### Writer collaborators

In [90]:
train['writer_collaborators'] = train.writer_collaboration_paths.apply(lambda x: number_of_occurences(x, 1))

In [101]:
(train.writer_collaborators > 0).value_counts()

False    573087
True      42425
Name: writer_collaborators, dtype: int64

In [102]:
train[train.writer_collaborators > 0].category.value_counts()

1    41680
0      745
Name: category, dtype: int64

###### Writer collaboration_min_distance

In [105]:
train['writer_collaboration_min_distance'] = train.writer_collaboration_paths.apply(lambda x: np.min(x) if x is not None else np.inf)

In [106]:
train['writer_collaboration_mean_distance'] = train.writer_collaboration_paths.apply(lambda x: np.mean(x) if x is not None else np.inf)